## 6. Popularity-based Model

This is the baseline approach. We will assign the most N popular reviews to each gamer and compute specific accuracy metrics for recommender systems such as Precision@N, Recall@N, average precision with cutoff.

### Sorting top-N Popularity

Exploring columns to represent popularity of games

In [ ]:
# recommendations.groupby('app_id').is_recommended.agg(['mean', 'sum']).sort_values('mean', ascending=False).reset_index()

,app_id,mean,sum
0,757310,1.0,7
1,742360,1.0,1
2,824000,1.0,1
3,814370,1.0,9
4,212160,1.0,3
...,...,...,...
1262,307960,0.0,0
1263,594330,0.0,0
1264,382110,0.0,0
1265,604500,0.0,0


In [ ]:
# games.rating.unique()

array(['Very Positive', 'Positive', 'Mixed', 'Mostly Positive',
       'Overwhelmingly Positive', 'Negative', 'Mostly Negative',
       'Overwhelmingly Negative', 'Very Negative'], dtype=object)

In [ ]:
# ratings = ['Overwhelmingly Positive', 'Very Positive', 'Positive',  
#            'Mostly Positive', 'Mixed',  
#            'Overwhelmingly Negative', 'Very Negative', 'Negative', 
#            'Mostly Negative']

# rating_values = [5, 4, 3, 2, 1, -5, -4, -3, -2]  
# rating_map = dict(zip(ratings, rating_values))
# games['rating_numeric'] = games['rating'].map(rating_map)

In [ ]:
# Top_Popularity_Games = games.sort_values(['rating_numeric', 'positive_ratio', 'user_reviews'], ascending=False)

# TopN = 10
# Top_Popularity_Games = Top_Popularity_Games[['app_id', 'title', 'rating_numeric', 'positive_ratio', 'user_reviews']].head(TopN)
# Top_Popularity_Games

,app_id,title,rating_numeric,positive_ratio,user_reviews
2287,2206340,Aokana - Four Rhythms Across the Blue - EXTRA2,5,100,530
47789,431730,Aseprite,5,99,11608
674,1144400,Senren＊Banka,5,99,11410
13392,1055540,A Short Hike,5,99,10534
29131,1698960,Project Kat - Paper Lily Prologue,5,99,3582
48525,1260520,Patrick's Parabox,5,99,2367
2577,1307580,TOEM,5,99,2321
16587,1901370,Ib,5,99,2073
2764,1530140,Aventura Copilului Albastru și Urât,5,99,1997
44726,2291760,Papa's Freezeria Deluxe,5,99,1964


Define game popularity as the highest number of recommended reviews:

In [ ]:
## split the data by users to prevent data leakage
review_users = recommendations.user_id.unique()
train_users, test_users = train_test_split(review_users, test_size=0.2, random_state=RANDOM_SEED)

X_train = recommendations[recommendations.user_id.isin(train_users)] 
X_test = recommendations[recommendations.user_id.isin(test_users)]
print('X_train: {} rows from {} unique users'.format(len(X_train.app_id), len(train_users)))
print('X_test: {} rows from {} unique users'.format(len(X_test.app_id), len(test_users)))

X_train: 8001363 rows from 4633398 unique users
X_test: 1998637 rows from 1158350 unique users


In [ ]:
# Evaluate the accuracy of the recommender system using precision, recall, and average precision

def precision_at_n(recommendations, ground_truth, n):
    # Compute the precision@n
    recommended_items = set(recommendations[:n])
    relevant_items = set(ground_truth)
    return len(recommended_items.intersection(relevant_items)) / n

def recall_at_n(recommendations, ground_truth, n):
    # Compute the recall@n
    recommended_items = set(recommendations[:n])
    relevant_items = set(ground_truth)
    return len(recommended_items.intersection(relevant_items)) / len(relevant_items)

def average_precision_at_cutoff(recommendations, ground_truth, n):
    # Compute the average precision at cutoff
    precision_sum = 0
    relevant_count = 0

    for i, item in enumerate(recommendations[:n]):
        if item in ground_truth:
            relevant_count += 1
            precision_sum += precision_at_n(recommendations, ground_truth, i + 1)

    return precision_sum / min(n, len(ground_truth))

def average_precision(recommender, ground_truth):
    is_relevant = np.array([int(item in ground_truth) for item in recommender])
    p_at_k = is_relevant * np.cumsum(is_relevant) / (1 + np.arange(is_relevant.shape[0]))
    avg_precision = np.sum(p_at_k) / is_relevant.shape[0]

    return avg_precision    

In [ ]:
## recommendations based on the top N most reviewed games in test set:
N = 10
popularity = X_train[X_train.is_recommended].app_id.value_counts()[:N].reset_index()
popularity.columns = ['app_id', 'review_count']
top_popularity = popularity.app_id.tolist()

# ground truth for each user
popularity_recs = (
    X_test[X_test.is_recommended]
    .sort_values('hours',ascending=False)
    .groupby('user_id')['app_id'].agg(list)
    .reset_index()
)
popularity_recs.columns = ['user_id', 'ground_truth']

popularity_recs['recommender'] = popularity_recs.user_id.apply(lambda x: list(top_popularity)) 
popularity_recs['precision'] =  popularity_recs.ground_truth.apply(lambda g: precision_at_n(top_popularity, g, N))
popularity_recs['avg_precision'] =  popularity_recs.ground_truth.apply(lambda g: average_precision(top_popularity, g))

# print("Precision@{}: {:.2f}".format(N, precision))
# print("Recall@{}: {:.2f}".format(N, recall))
print("Mean Average Precision@{}: {:.2f}".format(N, popularity_recs.avg_precision.mean()))

popularity_recs.head(10)

NameError: name 'X_train' is not defined

In [ ]:
# Compute the popularity of each game based on the number of reviews
popularity = recommendations['app_id'].value_counts().reset_index()

# Sort the games by popularity in descending order
popularity.columns = ['app_id', 'review_count']
popularity = popularity.sort_values(by='review_count', ascending=False)

# Define the number of popular reviews to assign to each gamer
N = 10

# Create a dictionary to store the recommendations for each gamer
recommendations_dict = {}

# For each gamer, assign the N most popular reviews as recommendations
for gamer in recommendations['user_id'].unique():
    recommendations_dict[gamer] = list(popularity['app_id'].head(N))

In [ ]:
# Example usage:
gamer_id = 51580
ground_truth = [252490]  # List of games that the gamer actually liked

precision = precision_at_n(recommendations_dict[gamer_id], ground_truth, N)
recall = recall_at_n(recommendations_dict[gamer_id], ground_truth, N)
avg_precision = average_precision_at_cutoff(recommendations_dict[gamer_id], ground_truth, N)

print("Precision@{}: {:.2f}".format(N, precision))
print("Recall@{}: {:.2f}".format(N, recall))
print("Average Precision@{}: {:.2f}".format(N, avg_precision))

Precision@10: 0.10
Recall@10: 1.00
Average Precision@10: 1.00


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Feature Engineering: Create a bag-of-words representation of game descriptions
vectorizer = CountVectorizer()
game_descriptions = vectorizer.fit_transform(games_metadata["description"])

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(game_descriptions, game_descriptions)


KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
# Function to get recommended games based on input game
def get_recommendations(app_id, similarity_matrix,dataset, top_n=5):
    # Get the index of the game
    game_index = dataset[dataset["app_id"] == app_id].index[0]
    
    # Get the similarity scores of the game with all other games
    sim_scores = list(enumerate(similarity_matrix[game_index]))
    
    # Sort the games based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the top N most similar games (excluding the input game itself)
    top_games_indices = [i[0] for i in sim_scores[1:top_n+1]]
    top_games = top_games_indices
    
    return top_games

In [ ]:
get_recommendations(113020,cosine_sim,games_metadata)

## Content-based filtering

In [ ]:
matrix_games_tags.info()

NameError: name 'matrix_games_tags' is not defined

### Dimensionality Reduction

Principal Component Analysis is conducted to reduce the number of game tags.

In [ ]:
pca = PCA(n_components=len(matrix_games_tags.columns))
pca.fit(matrix_games_tags)
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

In [ ]:
# Finding the suitable value of k
k = np.argmax(cumulative_variance_ratio >= 0.95) + 1

print("The suitable value of k for PCA with a threshold of 0.95 is:", k)

# View cumulative explained variance ratio with threshold
plt.plot(range(1, len(cumulative_variance_ratio) + 1), cumulative_variance_ratio, marker='o')
plt.axhline(y=0.95, color='r', linestyle='--', label='Threshold: 0.95')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio by Number of Principal Components')
plt.grid(True)
plt.show()

# 

In [ ]:
N = 10
k = 250
# Dimensionality Reduction: Apply PCA
pca = PCA(n_components=k)  # Set the desired number of components, k
reduced_features = pca.fit_transform(matrix_games_tags)

# Convert the reduced_features array back to a DataFrame
reduced_matrix_games_tags = pd.DataFrame(reduced_features, index=matrix_games_tags.index)

# Content-based Filtering (Similarity)
# Calculate similarity matrix
similarity_matrix = cosine_similarity(reduced_matrix_games_tags)

In [ ]:
# Example on one game
selected_game_id = 12345  # Choose the game ID for which you want to find similar games
similarity_scores = similarity_matrix[selected_game_id]
similar_games_indices = similarity_scores.argsort()[::-1][:N]  # Select top N similar games

recommended_games = matrix_games_tags.iloc[similar_games_indices].index.tolist()